In [1]:
my_packages <- c("data.table", "tidyverse",
                 "Seurat", "SingleR")
pacman::p_load(char = my_packages)

tmp <- list()

# Load data

Ref

In [14]:
mat_ref <- Matrix::readMM("temp/ref_singler.mtx")

In [15]:
dim(mat_ref)

[1] 36159 33538

In [16]:
mat_ref_gene <- fread("temp/ref_scibet_gene.csv", header = F)$V1
mat_ref_barcode <- fread("temp/ref_scibet_cellbarcode.csv", header = F)$V1
mat_ref_label <- fread("temp/ref_scibet_label.csv", header = F)$V1

In [43]:
table(mat_ref_label)

mat_ref_label
          B         CD4         CD8          DC endothelial  epithelial 
       1480        3307        6609          87        1701        9466 
fibroblasts  macrophage        mast          NK   pericytes      plasma 
       2761        2969         391        3207         591        2191 
       Treg 
       1399 

In [18]:
rownames(mat_ref) <- mat_ref_barcode
colnames(mat_ref) <- mat_ref_gene

In [57]:
mat_ref <- t(mat_ref)

In [58]:
mat_ref[1:5,1:5]

5 x 5 sparse Matrix of class "dgTMatrix"
            AAACCTGAGTGCGTGA-1 AAACGGGAGCTGCCCA-1 AAAGATGAGTGGACGT-1
MIR1302-2HG                  .                  .                  .
FAM138A                      .                  .                  .
OR4F5                        .                  .                  .
AL627309.1                   .                  .                  .
AL627309.3                   .                  .                  .
            AAAGTAGCACGCCAGT-1 AAATGCCTCTATCCCG-1
MIR1302-2HG                  .                  .
FAM138A                      .                  .
OR4F5                        .                  .
AL627309.1                   .                  .
AL627309.3                   .                  .

In [20]:
max(mat_ref[,1])

[1] 5.868889

test

In [13]:
mat_test <- Matrix::readMM("temp/adata_hvg_singler.mtx")

In [44]:
mat_test_barcode <- fread("temp/adata_hvg_scibet_cellbarcode.csv", header = F)$V1
mat_test_gene <- fread("temp/adata_hvg_scibet_gene.csv", header = F)$V1
mat_test_cluster <- fread("temp/adata_leiden.csv", header = F)$V1

In [23]:
rownames(mat_test) <- mat_test_barcode
colnames(mat_test) <- mat_test_gene

In [51]:
mat_test <- t(mat_test)

In [53]:
mat_test[1:5,1:5]

5 x 5 sparse Matrix of class "dgTMatrix"
         AAAGAACCAATTGCAC-1 AAAGGATGTCCCTCAT-1 AAAGGTACATTCACCC-1
HES4                      .                  .                  .
ISG15                     .                  .                  .
TNFRSF18                  .                  .                  .
TNFRSF4                   .                  .                  .
MXRA8                     .                  .                  .
         AAAGTGACACCCAAGC-1 AAATGGAGTGACCGTC-1
HES4                      .                  .
ISG15                     .                  .
TNFRSF18                  .                  .
TNFRSF4                   .                  .
MXRA8                     .                  .

In [52]:
max(mat_test[,1])

[1] 5.50273

# SingleR

In [59]:
prd <- SingleR(test = mat_test, 
                      labels = mat_ref_label,
                      ref = mat_ref, 
                      clusters = mat_test_cluster, 
                      fine.tune = TRUE,
                      BPPARAM = BiocParallel::SerialParam())

In [60]:
prd

DataFrame with 15 rows and 5 columns
                            scores first.labels     tuning.scores      labels
                          <matrix>  <character>       <DataFrame> <character>
0   0.350358:0.497735:0.498371:...         Treg 0.662048:0.628529        Treg
1   0.348419:0.470380:0.519634:...           NK 0.676267:0.641293          NK
2   0.267120:0.226457:0.232818:...   epithelial 0.642304:0.353911  epithelial
3   0.406855:0.401655:0.409688:...       plasma 0.628460:0.564026      plasma
4   0.485358:0.372705:0.377346:...            B 0.635932:0.627876          DC
...                            ...          ...               ...         ...
10  0.244050:0.222462:0.222575:...   epithelial 0.587579:0.331581  epithelial
11  0.322345:0.333000:0.343655:...         mast 0.560949:0.426664        mast
12  0.414749:0.454722:0.496620:...          CD8 0.579267:0.564375         CD8
13  0.362127:0.298929:0.304515:...   epithelial 0.532428:0.453952  epithelial
14  0.309083:0.312625:0.304

In [63]:
re_df <- data.frame(
    cluster = mat_test_cluster
)

In [65]:
re_df$label <- plyr::mapvalues(re_df$cluster, 
                                             from = as.character(c(0:14)),
                                             to = prd$labels)
table(re_df$label)


        CD8          DC endothelial  epithelial fibroblasts  macrophage 
       5035       97188       15673      148383       44494       89881 
       mast          NK      plasma        Treg 
      11792      165960      102386      153568 

# Save

In [ ]:
fwrite(data.frame(label = re_df$label), file = "Output/02/singler_result.csv")